In [5]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import fileinput
import sys


INSERT_TOKEN = '{{+{0}//{1}+}}'
DELETE_TOKEN = '[-{0}//{1}-]'


def parse_annotation(line):
    position, error_type, correction, _, _, annotator = line.split('|||')
    start, end = [int(index) for index in position.split()]
    annotator = int(annotator)
    correction = correction.strip()
    return start, end, error_type, correction, annotator


def parse_m2(lines):
    sentence = ''
    # tokens = []
    edits = []

    for line in lines:
        line = line.strip()

        if line.startswith('S '):
            sentence = line[2:].strip()
            # tokens = sentence.split()
            edits = []
        elif line.startswith('A '):
            start, end, error_type, correction, _ = parse_annotation(line[2:])
            # original = ' '.join(tokens[start:end])
            edits.append((start, end, correction, error_type))
        elif sentence:
            yield sentence, edits
            sentence = ''
            edits = []

    if sentence:
        yield sentence, edits


def to_edit_token(before, after, error_type):
    edit_token = []
    if before:
        before = before.replace(' ','\u3000')
        edit_token.append(DELETE_TOKEN.format(before, error_type))
    if after:
        after = after.replace(' ','\u3000')
        edit_token.append(INSERT_TOKEN.format(after, error_type))
    return ''.join(edit_token)


def to_diff(sent, edits):
    tokens = sent.split()
    before = []
    for start, end, after, error_type in reversed(edits):
        before.clear()
        for i in reversed(range(start, end)):
            before.append(tokens.pop(i))
        before_str = ' '.join(reversed(before))
        edit_token = to_edit_token(before_str, after, error_type)
        tokens.insert(start, edit_token)
    return ' '.join(tokens)


def main(iterator):
    FCE = []
    for sent, edits in parse_m2(iterator):
        FCE.append(to_diff(sent, edits))
    return FCE

In [7]:
#Tmp = main(open('fce.md'))

In [10]:
main(open('untitled.txt'))

['DECEMBER {+-NONE-//noop+} 12TH',
 'PRINCIPAL MR . {+-NONE-//noop+} ROBERTSON',
 'DEAR SIR {+-NONE-//noop+} ,']

In [17]:
' '.join([i for i in 'DECEMBER {+-NONE-//noop+} 12TH'.replace('{+-NONE-//noop+}','').split(' ') if i])

'DECEMBER 12TH'

In [19]:
FCE =[]
for i in Tmp:
    sen = i.replace('{+-NONE-//noop+}','')
    new_sen = ' '.join([token for token in sen.split(' ') if token])
    FCE.append(new_sen)

In [8]:
#FCE = [i.replace('{+-NONE-//noop+}','').strip() for i in Tmp]

In [20]:
f = open('FCE_errant_diff.txt','w+')
for i in FCE:
    f.write(i+'\n')
f.close()